# Natural Language Processing

## Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [14]:
dataset = pd.read_csv('Amazon_Reviews.csv')

## Cleaning the texts

In [15]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, len(dataset)):
    review = re.sub('[^a-zA-Z]', ' ', dataset['review'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = [word for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ALISHA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
corpus = dataset["review"]

## Creating the Bag of Words model

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 5000)
vectorizer.fit(corpus)
X = vectorizer.transform(corpus).toarray()
y = dataset.iloc[:, -1].values

## Tree-based feature selection

In [30]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
print(X.shape)

clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X, y)
clf.feature_importances_  

model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X)
X_new.shape 

(15000, 5000)


(15000, 1008)

## Splitting the dataset into the Training set and Test set

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size = 0.20, random_state = 0)

## Training the SVM model on the Training set

In [32]:
from sklearn.svm import LinearSVC as algorithm
svc = algorithm()
svc.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

## Training the NB model on the Training set

In [9]:
from sklearn.naive_bayes import GaussianNB as algorithm
nvb = algorithm()
nvb.fit(X_train, y_train)

GaussianNB(priors=None)

## Training the KNN model on the Training set

In [10]:
from sklearn.neighbors import KNeighborsClassifier as algorithm
knn = algorithm()
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

## Training the RF Classification model on the Training set

In [11]:
from sklearn.ensemble import RandomForestClassifier as algorithm
rf = algorithm()
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

## Predicting the Test set results

In [33]:
#y_pred_nvb = nvb.predict(X_test)
y_pred_svc = svc.predict(X_test)
#y_pred_knn = knn.predict(X_test)
#y_pred_rf = rf.predict(X_test)
#print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [13]:
#print(np.concatenate((y_pred_svc.reshape(len(y_pred_svc),1), y_pred_nvb.reshape(len(y_pred_nvb),1), y_test.reshape(len(y_test),1)),1))

## Making the Confusion Matrix

In [34]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred_svc)
print(cm)
accuracy_score(y_test, y_pred_svc)

[[1284  228]
 [ 216 1272]]


0.852

## Saving the model

In [36]:
import pickle
pickle.dump(svc, open('model_svc', 'wb'))
pickle.dump(vectorizer, open('vectorizer', 'wb'))
# model = pickle.load(open('model', 'rb'))
# model.score(X_test, y_test)
# model.predict(X_test)
# vec = pickle.load(open('vectorizer', 'rb'))